<a href="https://colab.research.google.com/github/Adria100/clin_IQ/blob/main/4_Evaluation_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Evaluation

In [ ]:
!pip install rouge_score
!pip install bert-score python-Levenshtein nltk
!pip install spacy transformers scikit-learn rouge-score nltk textacy
!python -m spacy download en_core_web_md

In [ ]:
import json
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
from nltk.tokenize import sent_tokenize
from transformers import pipeline
from bert_score import score
from nltk.translate.meteor_score import meteor_score
from sklearn.metrics import f1_score
import pandas as pd
import Levenshtein
import nltk
from nltk.corpus import wordnet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Update file path


In [ ]:
nlp = spacy.load('en_core_web_md')

# Hugging Face pipeline for entailment (textual consistency)
entailment_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
nltk.download("wordnet")
nltk.download("omw-1.4")

For multi-hop evaluation we have looked at evaluation approaches used in literature:

-BERTScore, BLEU, METEOR  and Levenshtein Ratio
Lemesle, Q., Chevelu, J., Martin, P., Lolive, D., Delhay, A., & Barbot, N. (2025, January). Paraphrase Generation Evaluation Powered by an LLM: A Semantic Metric, Not a Lexical One. In Proceedings of the 31st International Conference on Computational Linguistics (pp. 8057-8087).

-Cosine Similarity
Patwardhan, A., Vaidya, V., & Kundu, A. (2024, October). Automated Consistency Analysis of LLMs. In 2024 IEEE 6th International Conference on Trust, Privacy and Security in Intelligent Systems, and Applications (TPS-ISA) (pp. 118-127). IEEE.

-G-Eval
Liu, Y., Iter, D., Xu, Y., Wang, S., Xu, R., & Zhu, C. (2023). G-eval: NLG evaluation using gpt-4 with better human alignment. arXiv preprint arXiv:2303.16634.

-ROUGE
Briman, M. K. H., & Yildiz, B. (2024). Beyond ROUGE: A comprehensive evaluation metric for abstractive summarization leveraging similarity, entailment, and acceptability. International Journal on Artificial Intelligence Tools. DOI: https://doi. org/10.1142 S.


In [ ]:
def evaluate_classification(references, predictions):
    """
    Calculate accuracy, precision, and recall for letter classification.

    Parameters:
        y_true (list): True labels
        y_pred (list): Predicted labels

    Returns:
        dict: A dictionary with accuracy, precision, and recall
    """
    return {
        'accuracy': accuracy_score(references, predictions),
        'precision': precision_score(references, predictions, average='macro', zero_division=0),
        'recall': recall_score(references, predictions, average='macro', zero_division=0)
    }
#BERTScore
def evaluate_bertscore(references, predictions, lang='en'):
    P, R, F1 = score(predictions, references, lang=lang)
    return {
        'precision': P.mean().item(),
        'recall': R.mean().item(),
        'f1': F1.mean().item()
    }
#METEOR
def evaluate_meteor(references, predictions):
    scores = []
    for ref, pred in zip(references, predictions):
        scores.append(meteor_score([ref], pred))
    return {
        'average_meteor': sum(scores) / len(scores)
    }
#Levenshtein
def evaluate_levenshtein(references, predictions):
    distances = []
    for ref, pred in zip(references, predictions):
        distance = Levenshtein.distance(ref, pred)
        distances.append(distance)
    avg_distance = sum(distances) / len(distances)
    return {
        'average_levenshtein_distance': avg_distance
    }
#Cosine Similarity
def compute_cosine_similarity(references, predictions):
        """
        Compute cosine similarity between two texts using their embeddings.
        """
        doc1 = nlp(references)
        doc2 = nlp(predictions)
        return cosine_similarity([doc1.vector], [doc2.vector])[0][0]

#G-Eval
def evaluate_reasoning_flow(references, predictions):
        """
        Evaluate coherence and reasoning flow (G-Eval metric).
        This uses cosine similarity between consecutive sentences.
        """
        sentences = sent_tokenize(predictions)
        flow_scores = []

        for i in range(1, len(sentences)):
            score = compute_cosine_similarity(sentences[i-1], sentences[i])
            flow_scores.append(score)

        # Return average reasoning flow score
        return np.mean(flow_scores) if flow_scores else 0

#BLEU
def compute_bleu(references, predictions):
    """
    Compute BLEU score between a reference and a prediction.

    Parameters:
        reference (str): The ground truth answer.
        prediction (str): The model's predicted answer.

    Returns:
        float: BLEU score (0 to 1)
    """
    # Tokenize by splitting on whitespace
    reference_tokens = [references.split()]
    prediction_tokens = predictions.split()

    # Smoothing helps avoid zero scores for short predictions
    smoothie = SmoothingFunction().method4

    return sentence_bleu(reference_tokens, prediction_tokens, smoothing_function=smoothie)

#ROUGE
def compute_rouge(references, predictions):
    """
    Compute ROUGE scores between a reference and a prediction.

    Parameters:
        reference (str): The ground truth answer.
        prediction (str): The model's predicted answer.

    Returns:
        dict: ROUGE-1, ROUGE-2, and ROUGE-L F1 scores
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(references, predictions)

    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    }


class EvaluationSuite():
    """
    Class that contains the evaluation scripts for each question type.
    """

    def evaluate_MC(self, predictions, ground_truth):
        """
        Evaluates the accuracy of multiple-choice predictions.

        This function compares a list of predicted answers (e.g., "A", "B", "C", etc.)
        against the corresponding ground-truth labels and computes overall accuracy.

        Args:
            predictions (List[str]): A list of predicted choices (e.g., ["B", "C", "A", ...]).
            ground_truth (List[str]): A list of correct choices (same format and length as predictions).

        Returns:
            dict: A dictionary containing:
                - 'accuracy' (float): Overall classification accuracy.
                - 'precision' (float): Macro-averaged precision across all classes.
                - 'recall' (float): Macro-averaged recall across all classes.
        """
        # Ensure predictions and ground truth have the same length
        assert len(predictions) == len(ground_truth), "Predictions and ground_truth must have the same length."

        # Normalize labels (strip whitespace, convert to uppercase)
        predictions = [p.strip().upper() for p in predictions]
        ground_truth = [g.strip().upper() for g in ground_truth]

        # Generate confusion matrix
        labels = sorted(list(set(ground_truth + predictions)))  # Get all unique class labels
        cm = confusion_matrix(ground_truth, predictions, labels=labels)

        # Display the matrix
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        disp.plot(cmap='Blues', xticks_rotation=45)
        plt.title("Confusion Matrix")
        plt.tight_layout()
        plt.show()

        # Return classification metrics
        return evaluate_classification(ground_truth, predictions)

    def evaluate_FT(self, predictions, ground_truth):
        assert len(predictions) == len(ground_truth)

        labels = sorted(list(set(ground_truth + predictions)))

        # Compute confusion matrix
        cm = confusion_matrix(ground_truth, predictions, labels=labels)

        # Display the confusion matrix using ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        disp.plot(cmap='Blues', xticks_rotation=45)
        plt.title("Confusion Matrix")
        plt.tight_layout()
        plt.show()  # Show the plot

        # Compute and return classification metrics
        return self.evaluate_classification(ground_truth, predictions)

    def evaluate_SA(self, predictions, ground_truth):
        assert len(predictions) == len(ground_truth), "Predictions and ground_truth must have the same length."

        BLEU_scores = []
        ROUGE_scores = []

        for idx, prediction in enumerate(predictions):
            if prediction == 'N/A':
                continue

            print("Ground truth: ", ground_truth[idx])
            print("Prediction: ", prediction)

            bleu_score = compute_bleu(ground_truth[idx], prediction)
            rouge_score = compute_rouge(ground_truth[idx], prediction)

            print("BLEU: ", bleu_score)
            print("ROUGE: ", rouge_score)

            BLEU_scores.append(bleu_score)
            ROUGE_scores.append(rouge_score)

        avg_bleu = sum(BLEU_scores) / len(BLEU_scores) if BLEU_scores else 0
        avg_rouge = sum(ROUGE_scores) / len(ROUGE_scores) if ROUGE_scores else 0

        return avg_rouge, avg_bleu

    def evaluate_MH(self,predictions, ground_truth):
        # Ensure that predictions and ground_truth have the same length
        assert len(predictions) == len(ground_truth), "Predictions and ground_truth must have the same length."

        labels = sorted(list(set(ground_truth + predictions)))
        BLEU_scores = []
        ROUGE_scores = []
        similarity_scores = []
        coherence_scores = []

        # Loop through predictions and calculate BLEU and ROUGE scores
        for idx, prediction in enumerate(predictions):
            if prediction == 'N/A':
                continue  # Skip 'N/A' predictions

            # Print the ground truth and predicted answers for debugging
            print("Ground truth: ", ground_truth[idx])
            print("Prediction: ", prediction)

            # Compute BLEU and ROUGE scores
            bleu_score = compute_bleu(ground_truth[idx], prediction)
            rouge_score = compute_rouge(ground_truth[idx], prediction)
            similarity = compute_cosine_similarity(ground_truth[idx], prediction)
            coherence = evaluate_reasoning_flow(prediction)

            # Print the scores
            print("BLEU: ", bleu_score)
            print("ROUGE: ", rouge_score)
            print(f"Average Cosine Similarity:" similarity)
            print(f"Average Coherence:" coherence)

            # Append scores to the lists
            BLEU_scores.append(bleu_score)
            ROUGE_scores.append(rouge_score)
            avg_bleu = sum(BLEU_scores) / len(BLEU_scores) if BLEU_scores else 0
            avg_rouge = sum(ROUGE_scores) / len(ROUGE_scores) if ROUGE_scores else 0

            return avg_rouge, avg_bleu